In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

# Setup
pose = mp.solutions.pose.Pose()
dataset_path = "cricket_shot"
output_path = "cricket_shot_angles"
os.makedirs(output_path, exist_ok=True)

# Helper
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else angle

def extract_landmark(lm, name):
    return [lm[mp.solutions.pose.PoseLandmark[name].value].x,
            lm[mp.solutions.pose.PoseLandmark[name].value].y]

# Process each video
for filename in os.listdir(dataset_path):
    if filename.endswith(".mp4") or filename.endswith(".MOV"):
        video_path = os.path.join(dataset_path, filename)
        cap = cv2.VideoCapture(video_path)
        angles = []
        frame_num = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (640, 360))
            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frame_num += 1

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark
                try:
                    rs = extract_landmark(lm, "RIGHT_SHOULDER")
                    re = extract_landmark(lm, "RIGHT_ELBOW")
                    rw = extract_landmark(lm, "RIGHT_WRIST")
                    rh = extract_landmark(lm, "RIGHT_HIP")
                    rk = extract_landmark(lm, "RIGHT_KNEE")
                    ra = extract_landmark(lm, "RIGHT_ANKLE")
                    le = extract_landmark(lm, "LEFT_EYE")
                    no = extract_landmark(lm, "NOSE")
                    reye = extract_landmark(lm, "RIGHT_EYE")

                    angles.append({
                        "frame": frame_num,
                        "elbow": calculate_angle(rs, re, rw),
                        "shoulder": calculate_angle(rh, rs, re),
                        "knee": calculate_angle(rh, rk, ra),
                        "body": calculate_angle(rs, rh, rk),
                        "face": calculate_angle(le, no, reye)
                    })
                except:
                    continue

        cap.release()

        # Save angles
        shot_name = os.path.splitext(filename)[0]
        df = pd.DataFrame(angles)
        df.to_csv(os.path.join(output_path, f"{shot_name}_angles.csv"), index=False)

        # Optional: print angle range summary
        if not df.empty:
            summary = {
                "elbow": (round(df["elbow"].min(), 1), round(df["elbow"].max(), 1)),
                "shoulder": (round(df["shoulder"].min(), 1), round(df["shoulder"].max(), 1)),
                "knee": (round(df["knee"].min(), 1), round(df["knee"].max(), 1)),
                "body": (round(df["body"].min(), 1), round(df["body"].max(), 1)),
                "face": (round(df["face"].min(), 1), round(df["face"].max(), 1)),
            }
            print(f"{shot_name} ➤ {summary}")


I0000 00:00:1743655359.605915  447240 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743655359.692999  447441 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743655359.707091  447438 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

# Setup paths
dataset_path = "cricket_shot"
output_path = "cricket_shot_angles"
os.makedirs(output_path, exist_ok=True)

# Initialize pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Angle calculation helper
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else angle

# Landmark extractor
def get_lm(lm, name):
    return [lm[mp_pose.PoseLandmark[name].value].x, lm[mp_pose.PoseLandmark[name].value].y]

# Loop through all subfolders and videos
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith((".mp4", ".mov")):
            video_path = os.path.join(root, file)
            print(f"Processing {video_path}")
            cap = cv2.VideoCapture(video_path)
            angles = []
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (640, 360))
                results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                frame_count += 1

                if results.pose_landmarks:
                    lm = results.pose_landmarks.landmark
                    try:
                        rs = get_lm(lm, "RIGHT_SHOULDER")
                        re = get_lm(lm, "RIGHT_ELBOW")
                        rw = get_lm(lm, "RIGHT_WRIST")
                        rh = get_lm(lm, "RIGHT_HIP")
                        rk = get_lm(lm, "RIGHT_KNEE")
                        ra = get_lm(lm, "RIGHT_ANKLE")
                        le = get_lm(lm, "LEFT_EYE")
                        no = get_lm(lm, "NOSE")
                        reye = get_lm(lm, "RIGHT_EYE")

                        angles.append({
                            "frame": frame_count,
                            "elbow": calculate_angle(rs, re, rw),
                            "shoulder": calculate_angle(rh, rs, re),
                            "knee": calculate_angle(rh, rk, ra),
                            "body": calculate_angle(rs, rh, rk),
                            "face": calculate_angle(le, no, reye)
                        })
                    except:
                        continue
            cap.release()

            # Save angles
            if angles:
                label = os.path.basename(root)  # e.g., 'Cover_Drive'
                base_name = os.path.splitext(file)[0]  # e.g., 'video1'
                df = pd.DataFrame(angles)
                save_path = os.path.join(output_path, f"{label}_{base_name}_angles.csv")
                df.to_csv(save_path, index=False)
                print(f"Saved to {save_path}")


I0000 00:00:1743655813.129532  447240 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1743655813.203860  453713 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743655813.225830  453713 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing cricket_shot/Defence/IMG_2457.MOV


KeyboardInterrupt: 

In [5]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

# Paths
dataset_path = "cricket_shot"
output_path = "cricket_shot_angles"
os.makedirs(output_path, exist_ok=True)

# MediaPipe Pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Helper to calculate angle between 3 points
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else angle

# Helper to get landmark coordinates
def get_lm(lm, name):
    return [lm[mp_pose.PoseLandmark[name].value].x, lm[mp_pose.PoseLandmark[name].value].y]

# Go through all subfolders and video files
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith((".mp4", ".mov")):
            video_path = os.path.join(root, file)
            print(f"📽️ Processing: {video_path}")
            cap = cv2.VideoCapture(video_path)
            angles = []
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame = cv2.resize(frame, (640, 360))
                results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                frame_count += 1

                if results.pose_landmarks:
                    lm = results.pose_landmarks.landmark
                    try:
                        rs = get_lm(lm, "RIGHT_SHOULDER")
                        re = get_lm(lm, "RIGHT_ELBOW")
                        rw = get_lm(lm, "RIGHT_WRIST")
                        rh = get_lm(lm, "RIGHT_HIP")
                        rk = get_lm(lm, "RIGHT_KNEE")
                        ra = get_lm(lm, "RIGHT_ANKLE")
                        le = get_lm(lm, "LEFT_EYE")
                        no = get_lm(lm, "NOSE")
                        reye = get_lm(lm, "RIGHT_EYE")

                        angles.append({
                            "frame": frame_count,
                            "elbow": calculate_angle(rs, re, rw),
                            "shoulder": calculate_angle(rh, rs, re),
                            "knee": calculate_angle(rh, rk, ra),
                            "body": calculate_angle(rs, rh, rk),
                            "face": calculate_angle(le, no, reye)
                        })
                    except:
                        continue
            cap.release()

            # Save to proper subfolder
            if angles:
                shot_type = os.path.basename(root)  # e.g., Cover_Drive
                base_name = os.path.splitext(file)[0]  # e.g., video1
                save_dir = os.path.join(output_path, shot_type)
                os.makedirs(save_dir, exist_ok=True)

                save_path = os.path.join(save_dir, f"{base_name}_angles.csv")
                df = pd.DataFrame(angles)
                df.to_csv(save_path, index=False)
                print(f"✅ Saved: {save_path}")


I0000 00:00:1743655859.495341  447240 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1743655859.599909  454256 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743655859.619987  454256 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


📽️ Processing: cricket_shot/Defence/IMG_2457.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2457_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2443.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2443_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2442.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2442_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2456.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2456_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2468.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2468_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2440.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2440_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2454.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2454_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2455.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2455_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2441.MOV
✅ Saved: cricket_shot_angles/Defence/IMG_2441_angles.csv
📽️ Processing: cricket_shot/Defence/IMG_2469.M

In [6]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Paths
data_folder = 'cricket_shot_angles'
sequence_length = 60  # Max frames per video
angle_features = ['elbow', 'shoulder', 'knee', 'body', 'face']

X, y = [], []

# Step 1: Read angle sequences and labels
for shot_type in os.listdir(data_folder):
    shot_path = os.path.join(data_folder, shot_type)
    if not os.path.isdir(shot_path):
        continue
    for csv_file in os.listdir(shot_path):
        if csv_file.endswith('.csv'):
            file_path = os.path.join(shot_path, csv_file)
            df = pd.read_csv(file_path)
            if df.shape[0] < 5:
                continue  # skip tiny files

            # Step 2: Select angle columns and pad/truncate to 60 frames
            angles_seq = df[angle_features].values
            if angles_seq.shape[0] > sequence_length:
                angles_seq = angles_seq[:sequence_length]
            elif angles_seq.shape[0] < sequence_length:
                pad = np.zeros((sequence_length - angles_seq.shape[0], len(angle_features)))
                angles_seq = np.vstack([angles_seq, pad])

            X.append(angles_seq)
            y.append(shot_type)

# Step 3: Convert to arrays
X = np.array(X)  # shape: (num_samples, 60, 5)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Step 4: Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Step 5: LSTM model
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(sequence_length, len(angle_features))))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Step 6: Train
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=16)

# Step 7: Save model and label encoder
model.save("lstm_shot_classifier.h5")
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("✅ LSTM model and label encoder saved!")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 60, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        68,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,189 (301.52 KB)

 Trainable params: 77,189 (301.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.3003 - loss: 1.5635 - val_accuracy: 0.6190 - val_loss: 1.2952
Epoch 2/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6580 - loss: 1.2460 - val_accuracy: 0.6905 - val_loss: 1.1391
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7348 - loss: 1.0839 - val_accuracy: 0.7381 - val_loss: 1.0327
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7165 - loss: 0.9921 - val_accuracy: 0.6667 - val_loss: 0.9455
Epoch 5/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6778 - loss: 0.9550 - val_accuracy: 0.7857 - val_loss: 0.8440
Epoch 6/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7197 - loss: 0.8245 - val_accuracy: 0.7381 - val_loss: 0.7701
Epoch 7/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7520 - loss: 0.7105 - val_accuracy: 0.7857 - val_loss: 0.7358
Epoch 8/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8058 - loss: 0.6018 - val_accuracy: 0.7857 - v

✅ LSTM model and label encoder saved!


In [9]:
import os
import pandas as pd
from collections import defaultdict

# Path to your angle CSV folders
angle_dir = "cricket_shot_angles"

# To store cumulative values
angle_sums = defaultdict(lambda: {'elbow': [], 'shoulder': [], 'knee': [], 'body': [], 'face': []})

# Loop through each shot type folder
for shot_type in os.listdir(angle_dir):
    shot_folder = os.path.join(angle_dir, shot_type)
    if not os.path.isdir(shot_folder):
        continue

    for csv_file in os.listdir(shot_folder):
        if csv_file.endswith(".csv"):
            df = pd.read_csv(os.path.join(shot_folder, csv_file))
            if len(df) == 0:
                continue

            # Compute average for this video
            avg_angles = df[['elbow', 'shoulder', 'knee', 'body', 'face']].mean()

            # Append to shot type's angle list
            for key in avg_angles.keys():
                angle_sums[shot_type][key].append(avg_angles[key])

# Create final average table
summary_data = {
    "Shot Type": [],
    "Elbow": [],
    "Shoulder": [],
    "Knee": [],
    "Body": [],
    "Face": []
}

for shot_type, values in angle_sums.items():
    summary_data["Shot Type"].append(shot_type)
    summary_data["Elbow"].append(round(pd.Series(values["elbow"]).mean(), 2))
    summary_data["Shoulder"].append(round(pd.Series(values["shoulder"]).mean(), 2))
    summary_data["Knee"].append(round(pd.Series(values["knee"]).mean(), 2))
    summary_data["Body"].append(round(pd.Series(values["body"]).mean(), 2))
    summary_data["Face"].append(round(pd.Series(values["face"]).mean(), 2))

# Convert to DataFrame
summary_df = pd.DataFrame(summary_data)

# Save or display
summary_df.to_csv("average_shot_angles.csv", index=False)
print(summary_df)


     Shot Type   Elbow  Shoulder    Knee    Body   Face
0      Defence  119.50     28.02  120.82   95.43  84.11
1     Cut_shot   90.33     69.92   92.65   90.49  54.96
2  Cover_Drive  104.09     84.16  109.11  105.38  63.98
3    Pull_Shot   85.16     66.60  127.58  129.05  53.78
4   Sweep_Shot  117.42     50.02  104.57   99.62  68.40


In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

# Load model and label encoder
model = load_model("lstm_shot_classifier.h5")
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Define angle columns and sequence length
angle_features = ['elbow', 'shoulder', 'knee', 'body', 'face']
sequence_length = 60

def predict_shot_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    angles_seq = df[angle_features].values

    # Pad or truncate to fixed length
    if angles_seq.shape[0] > sequence_length:
        angles_seq = angles_seq[:sequence_length]
    elif angles_seq.shape[0] < sequence_length:
        pad = np.zeros((sequence_length - angles_seq.shape[0], len(angle_features)))
        angles_seq = np.vstack([angles_seq, pad])

    # Reshape for prediction: (1, 60, 5)
    angles_seq = np.expand_dims(angles_seq, axis=0)
    prediction = model.predict(angles_seq)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])[0]

    return predicted_label


In [8]:
from tensorflow.keras.models import load_model
import pickle

model = load_model("lstm_shot_classifier.h5")
with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

print("✅ Model and Label Encoder loaded successfully!")


✅ Model and Label Encoder loaded successfully!
